# Deep Reinforcement Learning Tutorial

https://www.youtube.com/watch?v=cO5g5qLrLSo&ab_channel=NicholasRenotte

### Install Dependencies

In [31]:
!pip install tensorflow==2.14.0
!pip install gym
!pip install keras
#!pip install keras-rl2
#!pip install gym[classic_control]

download visual studio

In [14]:
# ! pip install pyproject-toml
# ! pip install setuptools wheel
# ! pip install manimlib
# ! pip install manimce
# ! pip install --upgrade keras
! pip install keras-utils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2654 sha256=1b1a515224d88b9b3639434a097008ff9e479f4072e13cb8d8cee11a0381be78
  Stored in directory: c:\users\zuoch\appdata\local\pip\cache\wheels\84\04\c8\f3d21e09aa3a1e25bc4a4fc07341ca073d7372f33dbd344a06
Successfully built keras-utils


In [41]:
# ! pip install pygame
! pip install keras-rl2
# ! pip install --upgrade rl

     ---------------------------------------- 0.0/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     -------------- ----------------------- 20.5/52.1 kB 217.9 kB/s eta 0:00:01
     ----------------------------- -------- 41.0/52.1 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 52.1/52.1 kB 266.7 kB/s eta 0:00:00


In [34]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [69]:
! pip install keras-rl2==1.0.4
! tensorflow>=2.1.0,<2.2.0

The system cannot find the file specified.


### Test Random Environment with OpenAI Gym

In [83]:
import gym
import random

In [94]:
env = gym.make("CartPole-v0", render_mode="rgb_array")
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\Users\zuoch\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [95]:
print(states, actions)

4 2


In [96]:
episodes = 10
for episode in  range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info, _ = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

c:\Users\zuoch\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:34.0
Episode:2 Score:20.0
Episode:3 Score:16.0
Episode:4 Score:55.0
Episode:5 Score:19.0
Episode:6 Score:30.0
Episode:7 Score:19.0
Episode:8 Score:14.0
Episode:9 Score:13.0
Episode:10 Score:98.0


### Create a Deep Learning Model with Keras

In [149]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,Activation,Dropout,Flatten

In [164]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

In [165]:
def build_model(states, actions):  # states = 4, actions = 2
    model = tf.keras.Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation = "relu"))
    model.add(Dense(24, activation = "relu"))
    model.add(Flatten())
    model.add(Dense(actions, activation = "linear"))
    return model

In [166]:
model = build_model(states, actions)

In [167]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [168]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (None, 4)                 0         
                                                                 
 dense_36 (Dense)            (None, 24)                120       
                                                                 
 dense_37 (Dense)            (None, 24)                600       
                                                                 
 flatten_18 (Flatten)        (None, 24)                0         
                                                                 
 dense_38 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [192]:
print(list(model.output.shape))
print(list((None, actions)))
print(list(model.output.shape) == list((None, actions)))
print(list(model.output.shape) != list((None, actions)))

AttributeError: 'list' object has no attribute 'shape'

In [186]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [224]:
num_hidden = 24
inputs = layers.Input(shape=(states,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(actions, activation="linear")(common)
# critic = layers.Dense(1)(common)

model2 = keras.Model(inputs=inputs, outputs=action)

In [230]:
model2.summary()
print(model2.output)
print(action.shape)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_54 (Dense)            (None, 24)                120       
                                                                 
 dense_55 (Dense)            (None, 2)                 50        
                                                                 
Total params: 170 (680.00 Byte)
Trainable params: 170 (680.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Tensor("dense_55/BiasAdd:0", shape=(None, 2), dtype=float32)
(None, 2)


### Build Agent with Keras-RL

In [207]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Delete from keras.utils.generic_utils import Progbar, to from keras.utils import Progbar in callbacks.py

In [226]:
def build_plicy_memory(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    # dqn = DQNAgent(model = model, memory = memory, policy = policy, 
    #                nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    return policy, memory

In [227]:
policy, memory = build_plicy_memory(model, actions)

In [231]:
dqn = DQNAgent(model = model2, memory = memory, policy = policy, 
                nb_actions = action.shape[1], nb_steps_warmup = 10, target_model_update = 1e-2)

TypeError: len is not well defined for a symbolic Tensor (dense_55/BiasAdd:0). Please call `x.shape` rather than `len(x)` for shape information.

In [124]:
dqn.compile(Adam(lr = 1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.